In [2]:
import geopandas as gpd
import os
import xarray as xr
import numpy as np
import pandas as pd
import copy
import pickle
import gc
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
#import leafmap
import leafmap.foliumap as leafmap
from shapely.geometry import mapping
import pyproj
import folium

# sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [129]:
from tsai.all import *

In [3]:
data_dir = '/mnt/CEPH_PROJECTS/sao/openEO_Platform'

In [4]:
pv_geoms = gpd.read_file(f"{data_dir}/old_data/data/shapefiles/photovoltaic.shp")

In [5]:
threshold_area = 5000 #m^2 ~50mx100m
pv_geoms_32632 = pv_geoms.to_crs(32632)
big_pv_geoms_32632 = pv_geoms_32632.where(pv_geoms_32632["geometry"].area > threshold_area).dropna()
big_pv_geoms = big_pv_geoms_32632.to_crs(4326)
print(f"Number of selected PV Farms: {len(big_pv_geoms)}")

Number of selected PV Farms: 43


In [6]:
bands = ['B01','B02','B03','B04',
         'B05','B06','B07','B08',
         'B8A', 'B11', 'B12']

### Utils function

In [60]:
def json_s2_boundary(coords):
    bounds = (
        float(coords['x'].min()),  # minx
        float(coords['y'].min()),  # miny
        float(coords['x'].max()),  # maxx
        float(coords['y'].max())   # maxy
    )
    
    # Define the bounds
    minx, miny, maxx, maxy = bounds
    
    # Define the coordinates for the rectangle
    rectangle_coords = [
        [ minx, miny],
        [maxx,miny],
        [maxx, maxy],
        [minx, maxy],
        [minx, miny]  
        
    ]
    
    # Create a GeoJSON object for the rectangle
    rectangle_geojson = {
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [rectangle_coords]
                },
                "properties": {}
            }
        ]
    }
    
    return rectangle_geojson

In [165]:
def extract_s2_data(farm_id):
    start_date = '2022-01-01'
    end_date = '2022-12-31'
    
    all_pixels_ts = []
    
    geom = big_pv_geoms.iloc[farm_id]
    farm_id = geom.name
    try:
        data = xr.open_dataset(f"{data_dir}/old_data/data/netcdfs/S2_2022_{farm_id}.nc",decode_coords="all")
        coords = data.rio.reproject("EPSG:4326").coords
        s2_boundaries = json_s2_boundary(coords)
        
        cloud_mask = np.bitwise_or((data.SCL == 8),(data.SCL == 9))
        geodf = gpd.GeoDataFrame(geometry=[geom["geometry"]],crs="EPSG:32632")
        data = data.where(~cloud_mask)  

        ## This step to fill the gaps (nan values) spatially
        data = data.groupby('t').apply(lambda x: x.fillna(x.mean(dim=('x','y'))))
        data.rio.write_crs("epsg:32632", inplace=True) #32633


        ## Linear interpolation to have full time-series
        daily_date_range = pd.date_range(start=pd.to_datetime(start_date), 
                                         end=pd.to_datetime(end_date),
                                         freq='D')

        ds_daily = data.reindex(t=daily_date_range)
        ds_daily_interp = ds_daily.interpolate_na(dim='t', method='linear')
        
        #clipped = ds_daily_interp.rio.clip(geodf.geometry.values, geodf.crs, drop=False, invert=False)

        df = ds_daily_interp.to_dataframe().unstack(level='t')

        arr_ts= []
        for b in bands:
            df_b = df[b]
            #df_b = df_b.dropna(how = 'all')
            arr_ts.append(np.array(df_b))

        all_pixels_ts.append(np.stack(arr_ts, axis=1)) 
        te_samples = np.vstack(all_pixels_ts)
        te_samples = np.nan_to_num(te_samples, nan=-999999)

        
        #te_samples = te_samples.reshape(te_samples.shape[0], te_samples.shape[1]*te_samples.shape[2])
        df_normal = df_b.reset_index()
        x_coords = df_normal[['x', 'y']].values[:,0]
        y_coords = df_normal[['x', 'y']].values[:,1]


        return te_samples, x_coords, y_coords, s2_boundaries
    
    except:
        print(f"This farm : {farm_id} is not exist.")  

In [173]:
def temporal_metric(stat_name, farm_id,  invert=False):
    start_date = '2022-02-01'
    end_date = '2022-11-30'
    
    all_pixels_ts = []
    #Keep 10% out to use them for testing 
    geom = big_pv_geoms.iloc[farm_id]
    farm_id = geom.name
    
    try:
        data = xr.open_dataset(f"{data_dir}/old_data/data/netcdfs/S2_2022_{farm_id}.nc",decode_coords="all")
        coords = data.rio.reproject("EPSG:4326").coords
        s2_boundaries = json_s2_boundary(coords)
        
        cloud_mask = np.bitwise_or((data.SCL == 8),(data.SCL == 9))
        geodf = gpd.GeoDataFrame(geometry=[geom["geometry"]],crs="EPSG:32632")
        data = data.where(~cloud_mask)  

        ## This step to fill the gaps (nan values) spatially
        data = data.groupby('t').apply(lambda x: x.fillna(x.mean(dim=('x','y'))))
        data.rio.write_crs("epsg:32633", inplace=True) #32633

        # Filter the dataset based on time [Feb, Nov]
        filtered_ds = data.sel(t=slice(start_date, end_date))

        # Weekly mean 
        ds_monthly = filtered_ds.resample({"t": "M"}).apply(statistics_functions[stat_name])

        # Linear interpolate for missing values 
        ds_monthly_interp = ds_monthly.interpolate_na(dim='t', method='linear')

        # Crop the farm 
        #clipped = ds_monthly_interp.rio.clip(geodf.geometry.values, geodf.crs, drop=False, invert=invert)

        #Extract the monthly mean 
        df = ds_monthly_interp.to_dataframe().unstack(level='t')
        arr_ts= []
        for b in bands:
            df_b = df[b]
            #df_b = df_b.dropna(how = 'all')
            arr_ts.append(np.array(df_b))

        all_pixels_ts.append(np.stack(arr_ts, axis=1))   
        te_samples = np.vstack(all_pixels_ts)
  
        df_normal = df_b.reset_index()
        x_coords = df_normal[['x', 'y']].values[:,0]
        y_coords = df_normal[['x', 'y']].values[:,1]
        
        
        return te_samples, x_coords, y_coords, s2_boundaries

    except:
        print(f"This farm : {farm_id} is not exist.") 
            

In [133]:
def farm_plot(farm_id, s2_boundaries):
    geom_wgs84 = big_pv_geoms.iloc[farm_id]
    centroid = geom_wgs84.geometry.centroid
    
    # Add the PV farm boundaries
    ## Convert the polygon boundary to GeoJSON format
    geojson_farm = mapping(geom_wgs84.geometry)    
    m = leafmap.Map(center=[centroid.y, centroid.x], zoom=15)
    m.add_xyz_service("qms.EOX::Maps - Sentinel-2 cloudless")
    folium.GeoJson(geojson_farm).add_to(m)
    
    # Add s2 boundaries
    folium.GeoJson(s2_boundaries).add_to(m)
    
    return m, geojson_farm

In [163]:
def map_predictios(te_pred,x_coords, y_coords, geojson_farm, s2_boundaries):
    # Define the original CRS 
    original_crs = pyproj.CRS("EPSG:32633")#6326  32633
    
    # Define the target CRS (WGS84)
    target_crs = pyproj.CRS("EPSG:4326")
    
    # Create a pyproj transformer to perform the coordinate transformation
    transformer = pyproj.Transformer.from_crs(original_crs, target_crs, always_xy=True)
    
    # Transform coordinates from the original CRS to WGS84
    lon, lat = transformer.transform(x_coords, y_coords)
    
    # Create a map centered at the mean of coordinates
    min_x, max_x = np.min(lon), np.max(lon)
    min_y, max_y = np.min(lat), np.max(lat)
    center = [(min_y + max_y) / 2, (min_x + max_x) / 2]
    m = leafmap.Map(location=center, zoom_start=17)
    m.add_xyz_service("qms.EOX::Maps - Sentinel-2 cloudless")
    
    # Add farm boundaries
    folium.GeoJson(geojson_farm).add_to(m)
    
    # Add s2 boundaries
    folium.GeoJson(s2_boundaries).add_to(m)
    
    # Iterate over each point in the array
    for x, y, value in zip(lon, lat, te_pred):
        # Add a marker with color based on the value
        if value == 1:
            folium.CircleMarker(
                location=[y, x],  
                radius=5,
                color='red',
                fill=True,
                fill_color='red'
            ).add_to(m)
            
    return m

### Extract dataset 

In [148]:
# Input the farm id in the GDF
te_samples, x_coords, y_coords, s2_boundaries = extract_s2_data(farm_id = 9)
te_samples = te_samples.reshape(te_samples.shape[0], te_samples.shape[1]*te_samples.shape[2])
te_samples.shape

(380, 4015)

In [149]:
m, geojson_farm = farm_plot(farm_id = 9, s2_boundaries = s2_boundaries)
m

### Inference non temporal features

#### RF - 2D data

In [150]:
# Load the trained classifier 
clf = pickle.load(open(f'{data_dir}/s2/germany/models/non_temporal_models/rf_non_temporal_1.sav', 'rb'))

In [151]:
# Predictions 
te_pred = clf.predict(te_samples)

In [126]:
# custom_cmap = ListedColormap(['white', 'yellow']) #yellow for 1 and white for 0

# # Plot the binary mask
# plt.scatter(x_coords, y_coords, c=te_pred, cmap=custom_cmap, s=500)
# plt.xlabel('X')
# plt.ylabel('Y')
# plt.title('Binary Mask')
# plt.show()

In [152]:
map_predictios(te_pred,x_coords, y_coords, geojson_farm, s2_boundaries)

#### InceptionTime model - 3D 

In [154]:
InceptionTime_model = load_learner(f'{data_dir}/s2/germany/models/non_temporal_models/Inception_time_non_temporal_1.pkl', cpu=False)

In [155]:
te_samples, x_coords, y_coords, s2_boundaries = extract_s2_data(farm_id = 9)

In [161]:
probas, target, te_pred = InceptionTime_model.get_X_preds(te_samples)
te_pred2 = np.array(np.argmax(probas, axis=1))


In [164]:
map_predictios(te_pred2,x_coords, y_coords, geojson_farm, s2_boundaries)

### Inference - temporal features

In [ ]:
# Define a list of statistics functions
statistics_functions = {
    "mean": lambda x: x.mean(dim="t",skipna=True),
    "median": lambda x: x.median(dim="t",skipna=True),
    "std": lambda x: x.std(dim="t",skipna=True),
    # Add more statistics functions as needed
}

# Define the list of statistics names
statistics_names = list(statistics_functions.keys())

In [175]:
# Initialize a dictionary to store the results
results = {}

# Loop through each statistic
for stat_name in statistics_names:
    # Resample the dataset to monthly means using the current statistic function
    ds_monthly, x_coords, y_coords, s2_boundaries = temporal_metric(stat_name, farm_id=9, invert=True) #filtered_ds.resample({"t": "M"}).apply(statistics_functions[stat_name])
    # Store the result in the dictionary
    results[stat_name] = ds_monthly 
    

te_samples= np.concatenate(list(results.values()), axis=-1)
print(te_samples.shape)

(380, 11, 30)


#### RF - 2D data

In [177]:
clf = pickle.load(open(f'{data_dir}/s2/germany/models/temporal_models/rf_temporal_1.sav', 'rb'))

In [179]:
# Predictions 
te_samples1 = te_samples.reshape(te_samples.shape[0], te_samples.shape[1]*te_samples.shape[2])
te_pred = clf.predict(te_samples1)

In [180]:
map_predictios(te_pred,x_coords, y_coords, geojson_farm, s2_boundaries)

#### InceptionTime model - 3D

In [181]:

InceptionTime_model = load_learner(f'{data_dir}/s2/germany/models/temporal_models/Inception_time_temporal_1.pkl', cpu=False)

In [182]:
probas, target, te_pred = InceptionTime_model.get_X_preds(te_samples)
te_pred2 = np.array(np.argmax(probas, axis=1))

In [183]:
map_predictios(te_pred2,x_coords, y_coords, geojson_farm, s2_boundaries)